In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import glob
import pprint
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim
from collections import OrderedDict

from adapters import *
from dataset import NumpyCsvDataSet
from runner import ExptRunner
from networks.imageencoder import *
from networks.imagedecoder import *
from networks.DenseAutoEncoder import DenseAutoEncoder
from networks.composedautoencoder import ComposedAutoEncoder
from networks.ConvVae import ConvVae
from networks.vae import VAE
from networks.dense import *
from networks.lossfunctions import *
from networks.multinet import *
from networks.special import *
from helpers import *
import utils

In [3]:
%reset -f control_ce_loss_adapter
from networks.lossfunctions import control_ce_loss_adapter

Don't know how to reset  control_ce_loss_adapter, please run `%reset?` for details


In [4]:
import warnings
warnings.filterwarnings("ignore", category=torch.serialization.SourceChangeWarning)

In [5]:
opt = np.get_printoptions()
np.set_printoptions(threshold=np.inf)

In [6]:
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
print (device)

cpu


In [7]:
if torch.cuda.is_available():
    print (torch.cuda.current_device())
    torch.cuda.set_device(device)
    print (torch.cuda.current_device())

In [8]:
start = time.time()
data_folder = 'data/demoplanner/version1'
v1_train_data = NumpyCsvDataSet(data_folder + '/train.csv', device=device)
v1_test_data = NumpyCsvDataSet(data_folder + '/test.csv', device=device)
print ("Time Taken: {} sec".format(time.time() - start))

Time Taken: 36.17111396789551 sec


In [7]:
start = time.time()
data_folder = 'data/demoplanner/version2'
v2_train_data = NumpyCsvDataSet(data_folder + '/train.csv', device=device)
v2_test_data = NumpyCsvDataSet(data_folder + '/test.csv', device=device)
print ("Time Taken: {} sec".format(time.time() - start))

Time Taken: 283.2419309616089 sec


In [9]:
train_data = v1_train_data
test_data = v1_test_data

In [10]:
print (train_data.data.shape)
print (train_data.data.device)
print (train_data.data.dtype)

print (test_data.data.device)
print (test_data.data.shape)
print (test_data.data.dtype)

torch.Size([13280, 2058])
cpu
torch.float32
cpu
torch.Size([1341, 2058])
torch.float32


In [10]:
def load_pretrained(folder):
    chkptFile = 'runs/pretrained/' + folder + '/train_checkpoint.tar'
    return torch.load(chkptFile, map_location=device)['model']

pretrain_policy = Dense([4, 16, 16, 6], use_last_act=False, prefix='enc').to(device)
pretrain_policy.load_state_dict(torch.load('runs/pretrained/04-08-10-24-50-Policy-V2-Dense-CE/autoenc.pth', 
                                  map_location=device))

pretrain_dynamics = Dense([4, 16, 2], prefix='enc').to(device)
pretrain_dynamics.load_state_dict(torch.load('runs/pretrained/03-31-18-02-52-Dynamics-V1-Dense-SmoothL1/autoenc.pth', 
                                    map_location=device))

pretrain_imgEncDec = ComposedAutoEncoder(layers_channels=[16,16,16,16], useMaxPool=True, device=device).to(device)
pretrain_imgEncDec.load_state_dict(torch.load('runs/pretrained/03-28-14-10-20-DemoPl-V2-ComposedAutoEncoderWithMaxPool-SmoothL1-/autoenc.pth',
                               map_location=device))

pretrain_envEncDec = load_pretrained('04-20-11-03-59-EnvAutoEnc-V2-DenseAutoEncoder-MSE')
pretrain_ImgToEnvBaseLine = load_pretrained('04-21-12-51-48-ImageToEnvFunctionBaseLine-V2-3Conv2Dense-MSE')
latentModel0422 = load_pretrained('04-22-23-59-35-TriLossOnCombinedNet-V2-ImageEnvEncoder-MSE_Adam')
latentModel0427 = load_pretrained('04-27-21-54-52-QuadLossOnCombinedNet-V2-MultiNet-MSE')

pretrain_policy_augmented_y = load_pretrained('06-20-12-42-08-TrainPolicyUsingAugmentedY-V2-Dense-CE')

In [11]:
net_builder = lambda: MultiNet(policy=Dense([4, 16, 16, 6], use_last_act=False),
                               dynamics=Dense([4, 16, 2]))

In [27]:
img_layers = [16,16,16,16]
z_dim = 64 # This should be set based on above img_layers
env_layers = [4, 256, z_dim]
in_channels = 1
img_encoder = lambda: ImageEncoderFlatInput(in_channels, img_layers, 'imgenc', useMaxPool=True, addFlatten=True)
img_decoder = lambda: ImageDecoderFlatInput(z_dim, list(reversed(img_layers)), in_channels, 'imgdec', useSigmoid=True)
env_encoder = lambda: Dense(env_layers, use_last_act=False, prefix='envenc')
env_decoder = lambda: Dense(list(reversed(env_layers)), last_act='sigmoid', prefix='envdec')
combined = lambda l1, l2: nn.Sequential(OrderedDict([
                              ('EncNet', l1()),
                              ('DecNet', l2()),
                            ]))

In [20]:
def env_enc_pretrained_img_dec():
    net = combined(env_encoder, img_decoder).to(device)
    load_mapped_state_dict(net, pretrain_imgEncDec, 'DecNet.net.imgdec', 'decoder.net.dec')
    return net

def pretrained_img_enc_env_dec(env_decoder):
    net = combined(img_encoder, env_decoder).to(device)
    load_mapped_state_dict(net, pretrain_imgEncDec, 'EncNet.net.imgenc', 'encoder.net.enc')
    return net

def img_enc_env_dec_from_pretrained():
    net = combined(img_encoder, env_decoder).to(device)
    load_mapped_state_dict(net, pretrain_imgEncDec, 'EncNet.net.imgenc', 'encoder.net.enc')
    load_mapped_state_dict(net, pretrain_envEncDec, 'DecNet.net.envdec_fc0', 'decoder.dec_fc1') # Hacky
    return net

def multinet_with_pretrained_policy_and_dynamics():
    net = MultiNet(policy=Dense([4, 16, 16, 6], use_last_act=False, prefix='enc'),
                   dynamics=Dense([4, 16, 2], prefix='enc')).to(device)
    net.policy.load_state_dict(pretrain_policy.state_dict())
    net.dynamics.load_state_dict(pretrain_dynamics.state_dict())
    return net

def image_to_env(conv, dense):
    return nn.Sequential(OrderedDict([
        ('Conv', conv),
        ('Dense', dense)
    ])).to(device)

def end_to_end_net():
    ItoY = image_to_env(ImageEncoderFlatInput(1, [16,16,16], prefix='imgenc', useMaxPool=True, addFlatten=True),
                     Dense([256, 4])).to(device)
    policy = Dense([4, 16, 16, 6], use_last_act=False, prefix='pol').to(device)
    dynamics = Dense([4, 16, 2], prefix='dyn').to(device)
    net = EndToEndNet(ItoY=ItoY, policy=policy, dynamics=dynamics).to(device)
    return net

def end_to_end_net_from_pretrained(pretrain_indices=[], fix_indices=[]):
    net = end_to_end_net()
    if 1 in pretrain_indices:
        load_mapped_state_dict(net, pretrain_policy, 'policy.net.pol', 'net.enc')
    if 2 in pretrain_indices:
        load_mapped_state_dict(net, pretrain_dynamics, 'dynamics.net.dyn', 'net.enc')
    if 0 in pretrain_indices:
        load_mapped_state_dict(net, pretrain_ImgToEnvBaseLine, 'ItoY.Conv', 'Conv')
        load_mapped_state_dict(net, pretrain_ImgToEnvBaseLine, 'ItoY.Dense', 'Dense')
    net.configureTraining(filters=fix_indices)
    return net

def latent_policy_from_latent(latentReference, policy=None, fix_indices=[0]):
    latent = copy.deepcopy(latentReference)
    if not policy:
        policy = Dense([64, 16, 6], use_last_act=False, prefix='pol')
    net = LatentPolicyNet(latent=latent, policy=policy).to(device)
    net.configureTraining(filters=fix_indices)
    return net

def latent_policy_fixed_img_enc_pol_train_env_dec(latentFilter=None, policy=pretrain_policy, env_decoder=env_decoder):
    latent = pretrained_img_enc_env_dec(env_decoder)
    policy = copy.deepcopy(policy)
    configure_net_training(policy, False)
    configure_net_training(latent[0], False)
    return LatentPolicyNet(latent=latent, policy=policy, latentFilter=latentFilter).to(device)

def latentFilter_gt_x_latent_y(data, latent_output):
    return torch.cat((Xt_scaled_adapter(data), latent_output[:, 2:]), dim=1)

In [28]:
net = latent_policy_fixed_img_enc_pol_train_env_dec(latentFilter=latentFilter_gt_x_latent_y, policy=pretrain_policy_augmented_y)
net

LatentPolicyNet(
  (latent): Sequential(
    (EncNet): ImageEncoderFlatInput(
      (net): Sequential(
        (imgenc_conv0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (imgenc_relu0): ReLU()
        (imgenc_maxpool0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (imgenc_conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (imgenc_relu1): ReLU()
        (imgenc_maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (imgenc_conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (imgenc_relu2): ReLU()
        (imgenc_maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (imgenc_conv3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (imgenc_relu3): ReLU()
        (imgenc_maxpool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      

In [29]:
for p in net.parameters():
    print ('{}:{}'.format(p.size(), p.requires_grad))

torch.Size([16, 1, 3, 3]):False
torch.Size([16]):False
torch.Size([16, 16, 3, 3]):False
torch.Size([16]):False
torch.Size([16, 16, 3, 3]):False
torch.Size([16]):False
torch.Size([16, 16, 3, 3]):False
torch.Size([16]):False
torch.Size([256, 64]):True
torch.Size([256]):True
torch.Size([4, 256]):True
torch.Size([4]):True
torch.Size([16, 4]):False
torch.Size([16]):False
torch.Size([16, 16]):False
torch.Size([16]):False
torch.Size([6, 16]):False
torch.Size([6]):False


In [12]:
def run_expts(train_epochs):
    for i in range(len(expts)):
        arg_lists = expts[i]
        cons_args = arg_lists[0]
        train_args = {}
        if (len(arg_lists)) > 1:
            train_args = arg_lists[1]
        test_args = {}
        if (len(arg_lists)) > 2:
            test_args = arg_lists[2]
        
        runner = ExptRunner(train_data=train_data, test_data=test_data, device=device, **cons_args)
        print ("Experiment logs folder: {}".format(runner.log_folder))
        with open(runner.log_folder + '/expt.txt', 'w') as f:
            f.write(pprint.pformat(arg_lists))
        runner.train(train_epochs, **train_args)
        runner.test(**test_args)

In [20]:
expts = [
#     [{
#         "expt_prefix":'Fix_ImagEnc_AugmentedPolicy_Train_EnvDec-V2-LatentPolicyNet-CE',
#         "net_func": lambda: latent_policy_fixed_img_enc_pol_train_env_dec(latentFilter=latentFilter_gt_x_latent_y, policy=pretrain_policy_augmented_y),
#         "data_to_label_adapter": policy_groud_truth_class_adapter,
#         "loss_adapter_func": control_ce_loss_adapter,
#      },
#     {
#     },
#     {
#         "loss_adapter": policy_l1_loss_adapter
#     }],
    [{
        "expt_prefix":'Finetune-0616232643-V2-LatentPolicyNet-CE',
        "net_func": lambda: net,
        "data_to_label_adapter": policy_groud_truth_class_adapter,
        "loss_adapter_func": control_ce_loss_adapter,
     },
    {
    },
    {
        "loss_adapter": policy_l1_loss_adapter
    }],
#     [{
#         "expt_prefix":'TrainPolicyUsingAugmentedY-V2-Dense-CE',
#         "net_func": lambda: Dense([6, 16, 16, 6], use_last_act=False, prefix='pol'),
#         "data_adapter_func": Xt_XtYt_scaled_adapter,
#         "data_to_label_adapter": policy_groud_truth_class_adapter,
#         "loss_adapter_func": control_ce_loss_adapter,
#      },
#     {
#     },
#     {
#         "loss_adapter": policy_l1_loss_adapter
#     }],
#     [{
#         "expt_prefix":'InitAndFixLatentTrainPolicy-V2-LatentPolicyNet-CE',
#         "net_func": lambda: latent_policy_from_latent(latentModel0422.imgEnc, policy=Dense([64, 6], use_last_act=False, prefix='pol')),
#         "data_to_label_adapter": policy_groud_truth_class_adapter,
#         "loss_adapter_func": control_ce_loss_adapter,
#      },
#     {
#     },
#     {
#         "loss_adapter": policy_l1_loss_adapter
#     }],
]

In [ ]:
train_epochs = 150
run_expts(train_epochs)

In [ ]:
runner = ExptRunner('ForTestReport', net_func= lambda:net, train_data=train_data, test_data=test_data, device=device,
                    loss_adapter_func=None, data_adapter_func=chkpt['input_adapter'], data_to_label_adapter=chkpt['label_adapter'])
print ("Experiment logs folder: {}".format(runner.log_folder))
runner.test()

In [ ]:
def build_reporter(checkpoint_file, useL1Loss=True):
    rr = ReportResults(test_data.data, train_data.data, device,
                       checkpoint_file, useL1Loss)
    return rr

In [ ]:
def combined_net_report(checkpoint_file):
    cp = torch.load(checkpoint_file, map_location=device)
    net = cp['model']
    se = SpecialExptRunner('', net, None, test_data, device=device)
    se.eval_end_of_training()

In [ ]:
sample = torch.cat((test_data.data[1055:1060], test_data.data[1230:1235]), axis=0)
ground_truth = policy_groud_truth_class_adapter(sample).to(device)
for folder, net in rr.build_net(rootdir='runs/04-07/'):
    print ('Folder: ' + folder)
    op_batch, loss = rr.run_mini_batch(net, sample)
    x_loss = F.cross_entropy(op_batch[:, :3], ground_truth[:, 0], reduction='none')
    y_loss = F.cross_entropy(op_batch[:, 3:], ground_truth[:, 1], reduction='none')
    print (ground_truth)
    print (op_batch)
    print (loss)
    print (x_loss)
    print (y_loss)
    print ('-----------------------------------------')

In [ ]:
sample = torch.cat((test_data.data[1055:1060], test_data.data[1230:1235]), axis=0)
ground_truth = policy_groud_truth_adapter(sample).to(device)
for folder, net in rr.build_net(rootdir='runs/04-08/'):
    print ('Folder: ' + folder)
    op_batch, loss = rr.run_mini_batch(net, sample)
    print (ground_truth)
    print (op_batch)
    print (loss)

In [ ]:
np.set_printoptions(**opt)